In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')


In [ ]:
os.chdir(r'/content/gdrive/My Drive/House Prices/')


In [ ]:
import pandas as pd
df1 = pd.read_csv('naperville_2020_2.csv', encoding="latin-1")
df2 = pd.read_csv('naperville_2018_2.csv', encoding="latin-1")
df3 = pd.read_csv('naperville_2019_2.csv', encoding="latin-1")
df4 = pd.read_csv('bolingbrook_2020_2.csv', encoding="latin-1")
df5 = pd.read_csv('bolingbrook_2019_2.csv', encoding="latin-1")
df6 = pd.read_csv('bolingbrook_2018_2.csv', encoding="latin-1")
df7 = pd.read_csv('naperville_2021-2022_2.csv', encoding="latin-1")
df8 = pd.read_csv('bolingbrook_2021-2022_2.csv', encoding="latin-1")
df = df1.append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True).append(df5, ignore_index=True).append(df6, ignore_index=True).append(df7, ignore_index=True).append(df8, ignore_index=True)
df = df.replace(r'[<%]', '', regex=True)
df = df.drop_duplicates()
print(df.shape)


In [ ]:
df.mean()

In [ ]:
df.std()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df.dtypes

In [ ]:
df.dropna()

In [ ]:
df.describe()

In [ ]:
print(df.shape)

In [ ]:
df['housingprice'] = df['housingprice'].replace('[-\$,+]', '', regex=True).astype(float)
df = df[df['housingprice'] < 2500000]
#df = df.rename(columns={"housingprice": "House Price"})

df['beds'] = df['beds'].replace('[-\$,+]', '', regex=True)
df['beds'].replace('', np.nan, inplace=True)
df['beds'] = df['beds'].replace('[-\$,+]', '', regex=True).astype(float)

df['baths'] = df['baths'].replace('[-\$,+]', '', regex=True).astype(float)

df['sqft'] = df['sqft'].replace('[-\$,+]', '', regex=True).astype(float)
df = df[df['sqft'] < 10000]

df['Tax Annual Amount'] = df['Tax Annual Amount'].replace('[-\$,+]', '', regex=True).astype(float)
#df = df.rename(columns={"Tax Annual Amount": "Tax"})
df['Basement Sq. Ft.'] = df['Basement Sq. Ft.'].replace('[-\$,+]', '', regex=True).astype(float)
#df = df.rename(columns={"Basement Sq. Ft.": "Basement Sqft"})

df['# Of Cars'] = df['# Of Cars'].replace('[-\$,+]', '', regex=True).astype(float)
df["# Of Cars"] = df["# Of Cars"].fillna(0)
df = df[(df['# Of Cars'] < 4.5)]

df['Year Built'] = df['Year Built'].astype(float)

df['# Of Cars'] = df['# Of Cars'].replace(0, np.nan)
# df = df.rename(columns={"# Of Cars": "Car Spaces"})
# df = df.rename(columns={"# of Baths (Full)": "Full Baths"})
# df = df.rename(columns={"# of Baths (1/2)": "Half Baths"})
# df = df.rename(columns={"# of Rooms": "Rooms"})
# df = df.rename(columns={"# of carpet": "Carpet Rooms"})
# df = df.rename(columns={"# of hardwood": "Hardwood Rooms"})

df['Basement'] = df['Basement'].replace('(Full)', 'Full', regex=True)
df['Basement'] = df['Basement'].replace('(Partial)', 'Partial', regex=True)
df['Basement'] = df['Basement'].replace('(Full, Walkout)', 'Full', regex=True)
df['Basement'] = df['Basement'].replace('(Walkout)', 'Full', regex=True)
df['Basement'] = df['Basement'].replace('[,]', '', regex=True)
df['Basement'] = df['Basement'].replace('', np.nan)
df['Basement'] = df['Basement'].replace(np.nan, 'None')
#df['month'] = df['month'].astype(float)

def parse_year(x):
  if x == 'RKET':
    return np.nan  
  else:
    return x

#df['yearsold'] = df['yearsold'].apply(parse_year)
#df['yearsold'] = df['yearsold'].astype(float)

In [ ]:
first_column = df.pop('House Price')
df.insert(0, 'House Price', first_column)

In [ ]:
df['Property Type'] = df['Property Type'].replace('(Condo/Co-op)', 'Condo', regex=True)
def parse_property_type(x):
  if x in ['Single Family Residential', 'Condo', 'Townhouse']:
    return x
  elif "Attached" in x:
    return 'Condo'
  elif "Detached" in x:
    return 'Single Family Residential'
  else: 
    return 'Other'
df['Property Type'] = df['Property Type'].apply(parse_property_type)
df['Property Type'] = df['Property Type'].astype(str)
#df['# of Rooms'] = df['# of Rooms'].replace(0, np.nan)
df['# of carpet'] = df['# of carpet'].replace(0, np.nan)
df['# of hardwood'] = df['# of hardwood'].replace(0, np.nan)

In [ ]:
heatinglist = ['Baseboard','Natural Gas','Other']
housinglist = ['Single Family Residential', 'Townhouse', 'Condo', 'Other']
basementlist = ['Finished', 'Slab', 'None', 'Unfinished', 'Partially Finished', 'Crawl', 'Other']
coolinglist = ['Central Air', 'Zoned', 'Other']
basementlist2 = ['None', 'Full', 'Partial', 'Full English', 'English', 'Partial English', 'Partial Full']
# for all categorical variables we selected
def top_x(df2,variable,top_x_labels):
  for label in top_x_labels:
    df2[variable+'_'+str(label)] = np.where(df2[variable]==label,1,0)


In [ ]:
df['Heating'] = df['Heating'].replace(np.nan, '', regex=True)
df['Cooling'] = df['Cooling'].replace(np.nan, '', regex=True)
df['Basement Description'] = df['Basement Description'].replace(np.nan, '', regex=True)
for index,row in df.iterrows():
    try:
      if 'Natural Gas' in row['Heating']:
          df.at[index,'Heating'] = 'Natural Gas'
      elif 'Baseboard' in row['Heating']:
          df.at[index,'Heating'] = 'Baseboard'
      else:
          df.at[index,'Heating'] = 'Other'
    except:
      df.at[index,'Heating'] = 'Other'

for index,row in df.iterrows():
    try:
      if 'Finished' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'Finished'
      elif 'Partially Finished' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'Partially Finished'
      elif 'Unfinished' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'Unfinished'
      elif 'Slab' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'Slab'
      elif 'None' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'None'
      elif 'Crawl' in row['Basement Description']:
          df.at[index,'Basement Description'] = 'Crawl'
      else:
          df.at[index,'Basement Description'] = 'Other'
    except:
      df.at[index,'Basement Description'] = 'Other'

for index,row in df.iterrows():
    try:
      if 'Zoned' in row['Cooling']:
          df.at[index,'Cooling'] = 'Zoned'
      elif 'Central Air' in row['Cooling']:
          df.at[index,'Cooling'] = 'Central Air'
      else:
          df.at[index,'Cooling'] = 'Other'
    except:
      df.at[index,'Cooling'] = 'Other'

top_x(df,'Cooling',coolinglist)
top_x(df,'Heating',heatinglist)
top_x(df,'Property Type', housinglist)
top_x(df,'Basement Description', basementlist)
top_x(df,'Basement', basementlist2)

In [ ]:
#csv's for 2018 uses this format when parsing data
df['# of rooms'] = df['# of carpet'] + df['# of hardwood']
df['# of rooms'] = df['# of rooms'].replace(0, np.nan)

In [ ]:
df.head(100)

In [ ]:
print(df.shape)

In [ ]:
sns.distplot(df['housingprice']);


In [ ]:
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":df["housingprice"], "log(price + 1)":np.log1p(df["housingprice"])})
prices.hist()

In [ ]:
corr = df.corr()
corr.sort_values(['housingprice'], ascending = False, inplace = True)
print(corr.housingprice)

In [ ]:
df['housingprice_log'] = np.log1p(df['housingprice'])

In [ ]:
y.head(5)

In [ ]:
df_to_train.head(5)

In [ ]:
df_to_train.fillna(value=df_to_train.mean(),inplace=True)

In [ ]:
df_to_train.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
y.hist()

In [ ]:
#col_to_train = ['Property Type', 'Year Built','# Of Cars','beds','sqft','baths','# of hardwood','# of rooms', 'Basement', 'Basement Sq. Ft.','Tax Annual Amount']
col_to_train = [i for i in df.columns if i not in ['Property Type','HOA Dues', 'List Price', 'Est. Mo. Payment', 'High School', 'Community', 'address', 'Basement', 'Heating', 'Cooling', 'Basement Description', '# of Baths (1/2)', 'housingprice', 'longtitude', 'latitude']] 
print(col_to_train)
df_to_train = df[col_to_train]

['Year Built', '# Of Cars', 'beds', 'baths', 'sqft', '# of Baths (Full)', '# of carpet', '# of hardwood', 'Basement Sq. Ft.', 'Tax Annual Amount', 'Cooling_Central Air', 'Cooling_Zoned', 'Cooling_Other', 'Heating_Baseboard', 'Heating_Natural Gas', 'Heating_Other', 'Property Type_Single Family Residential', 'Property Type_Townhouse', 'Property Type_Condo', 'Property Type_Other', 'Basement Description_Finished', 'Basement Description_Slab', 'Basement Description_None', 'Basement Description_Unfinished', 'Basement Description_Partially Finished', 'Basement Description_Crawl', 'Basement Description_Other', 'Basement_None', 'Basement_Full', 'Basement_Partial', 'Basement_Full English', 'Basement_English', 'Basement_Partial English', 'Basement_Partial Full', '# of rooms']


In [ ]:
df_to_train.fillna(value=df_to_train.mean(),inplace=True)

In [ ]:
X = df_to_train
y = df['housingprice']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=69)

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

Linear Regression

In [ ]:
df_to_train

In [ ]:
reg = LinearRegression().fit(train_X, train_y)

In [ ]:
#Decision Tree/ Random Forest/ S

In [ ]:
y_train_pred = reg.predict(train_X)
y_test_pred = reg.predict(test_X)

In [ ]:
r2_score(test_y, y_test_pred)

In [ ]:
print("Absolute Error", mean_absolute_error(test_y,y_test_pred))
print("Squared Error", mean_squared_error(test_y, y_test_pred)**(0.5))


Support Vector Regression

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
svr_model = SVR(kernel='linear')
svr_model.fit(train_X,train_y)

SVR(kernel='linear')

In [ ]:
y_train_pred = svr_model.predict(train_X)
y_test_pred = svr_model.predict(test_X)

In [ ]:
r2_score(test_y, y_test_pred)

0.6520672581827405

In [ ]:
print("Absolute Error", mean_absolute_error(test_y,y_test_pred))
print("Squared Error", mean_squared_error(test_y, y_test_pred)**(0.5))


Absolute Error 59843.27158779008
Squared Error 105274.70735315104


Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
            

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# Fit the random search model
rf_random.fit(train_X, train_y)

In [ ]:
y_train_pred = rf_random.predict(train_X)
y_test_pred = rf_random.predict(test_X)

In [ ]:
r2_score(test_y, y_test_pred)

0.8796737229753421

In [ ]:
print("Absolute Error", mean_absolute_error(test_y,y_test_pred))
print("Squared Error", mean_squared_error(test_y, y_test_pred)**(0.5))

Absolute Error 49305.695598183316
Squared Error 99710.72477356247


DecisionTrees

In [ ]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'min_samples_leaf': [5, 6, 7, 9,11, 12, 13, 15]}
reg_decision_model = DecisionTreeRegressor(random_state = 42) 

In [ ]:
reg_decision_model.get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [ ]:
regressor = GridSearchCV(reg_decision_model,param_grid=param_grid,scoring='neg_mean_absolute_error',cv=3,verbose=3)
regressor.fit(train_X, train_y)

In [ ]:
y_train_pred = regressor.predict(train_X)
y_test_pred = regressor.predict(test_X)

In [ ]:
r2_score(test_y, y_test_pred)

In [ ]:
print("Absolute Error", mean_absolute_error(test_y,y_test_pred))
print("Squared Error", mean_squared_error(test_y, y_test_pred)**(0.5))

DATAPREP

In [ ]:
!pip install dataprep
from dataprep.eda import create_report
report = create_report(df)

In [ ]:
report.save('paper2')

Graphs


In [ ]:
var = 'sqft'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = '# of Baths (Full)'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = '# of hardwood'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = 'beds'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = 'baths'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = 'Tax Annual Amount'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = '# of rooms'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = '# Of Cars'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = 'Year Built'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = '# of hardwood'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
var = 'Basement Sq. Ft.'
data = pd.concat([df['housingprice'], df[var]], axis=1)
data.plot.scatter(x=var, y='housingprice', ylim=(10000,5000000));

In [ ]:
sns.swarmplot(x="High School", y="housingprice", data=df)


In [ ]:
sns.swarmplot(x="Basement", y="housingprice", data=df)


In [ ]:
sns.swarmplot(x="Community", y="housingprice", data=df)


In [ ]:
sns.swarmplot(x="Property Type", y="housingprice", data=df)


Coordinates


In [ ]:
from geopy.geocoders import Nominatim
#exception GeocoderTimedOut send excel file of coordiantes

def generate_latitude(x):
  geolocator = Nominatim(user_agent="Your_Name")
  location = geolocator.geocode(x.address, timeout=10)
  if location is not None:
    return location.latitude

geolocator = Nominatim(user_agent="Your_Name")
for i in range(len(df)):  
  try:  
    loc = geolocator.geocode(str(df['address'][i]))  
    try:  
      la=loc.latitude  
      lo=loc.longitude  
      #df.loc[i, 'coord_code']= 1  
    except:  
      pass  
  except: 
    lo = 'NA'
    la = 'NA' 
    print('Error Place Full Name')   
  df.loc[i, 'longtitude']= lo  
  df.loc[i, 'latitude']= la  

def generate_longitude(x):
  geolocator = Nominatim(user_agent="Your_Name")
  location = geolocator.geocode(x.address, timeout=10)
  if location is not None:
    return location.longitude

#df2['latitude'] = df2.apply(generate_latitude, 1)
#df2['longitude'] = df2.apply(generate_longitude, 1)


XgBoost

In [ ]:
!pip install shap

In [ ]:
#col_to_train = ['Property Type', 'Year Built','# Of Cars','beds','sqft','baths','# of hardwood','# of rooms', 'Basement', 'Basement Sq. Ft.','Tax Annual Amount']
col_to_train = [i for i in df.columns if i not in ['Property Type','HOA Dues', 'List Price', 'Est. Mo. Payment', 'High School', 'Community', 'address', 'Basement', 'Heating', 'Cooling', 'Basement Description', '# of Baths (1/2)', 'housingprice', 'longtitude', 'latitude']] 
print(col_to_train)
df_to_train = df_summer[col_to_train]

In [ ]:
import numpy as np
from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


import xgboost as xgb

X = df_to_train
y = df_summer['housingprice']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=69)


In [ ]:
xgb_model = xgb.XGBRegressor()

In [ ]:
parameters = {'nthread':[1, 10, 20], #when use hyperthread, xgboost may become slower
              'objective':['reg:gamma'],
              'learning_rate': [0.1], #so called `eta` value
              'max_depth': [4],
              'min_child_weight': [4],
              'n_estimators': [325],
              'subsample': [1],
              'gamma':[0],
              'alpha':[0]}
random_search = GridSearchCV(xgb_model,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=3)

random_search.fit(train_X, train_y)
best_regressor = random_search.best_estimator_

In [ ]:
pvt = pd.pivot_table(pd.DataFrame(random_search.cv_results_),values='mean_test_score',index='param_nthread',columns='param_max_depth')

In [ ]:
sns.heatmap(pvt)

In [ ]:
random_search.cv_results_

In [ ]:
params = {'nthread':[30, 35, 45],
          'max_depth': [5, 6],
          'learning_rate': [0.1],
          'subsample': [0.9],
          'objective':['reg:gamma'],
          'colsample_bytree': [0.9],
          'min_child_weight': [4, 5, 6],
          'colsample_bylevel': [0.8],
          'n_estimators': [300]}

clf = RandomizedSearchCV(estimator=xgb_model,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=3)
clf.fit(train_X, train_y)
best_regressor = clf.best_estimator_

In [ ]:
print(clf.best_estimator_)

In [ ]:
pvt = pd.pivot_table(pd.DataFrame(clf.cv_results_),values='mean_test_score',index='param_nthread',columns='param_max_depth')
sns.heatmap(pvt)

In [ ]:
#save regressor as an object
best_regressor = random_search.best_estimator_

In [ ]:
#save model
import pickle
filename = 'finalized_model_2019.pkl'
pickle.dump(best_regressor, open(filename, 'wb'))

In [ ]:
#load model
import pickle
best_regressor = pickle.load(open('finalized_model.pkl', 'rb'))

In [ ]:
#large dataset error
from sklearn.metrics import mean_absolute_error

y_pred = best_regressor.predict(test_X)
abs_pred = mean_absolute_error(test_y, y_pred)
root_pred = mean_squared_error(test_y, y_pred)  

print("Root Mean Absolute Error:" , abs_pred)
print("Root Error:" , root_pred ** (0.5))


In [ ]:
y_pred = best_regressor.predict(train_X)
abs_pred = mean_absolute_error(train_y, y_pred)
root_pred = mean_squared_error(train_y, y_pred)  
print("Root Mean Absolute Error:" , abs_pred)
print("Root Error:" , root_pred ** (0.5))

In [ ]:
best_regressor.score(test_X, test_y)

In [ ]:
#singular error
from sklearn.metrics import mean_absolute_error

col_to_train = [i for i in df.columns if i not in ['Property Type','HOA Dues', 'List Price', 'Est. Mo. Payment', 'High School', 'Community', 'address', 'Basement', 'Heating', 'Cooling', 'Basement Description', '# of Baths (1/2)', 'housingprice', 'longtitude', 'latitude']] 
df_to_train = df[col_to_train]
y_pred = best_regressor.predict(df_to_train)
rmse_pred = mean_absolute_error(df['housingprice'], y_pred) 

print("Root Mean Absolute Error:" , rmse_pred)
print("prediction:", y_pred)
print("last price:", df["housingprice"][0])



SHAP


In [ ]:
import shap
explainer = shap.TreeExplainer(best_regressor)
shap_values = explainer.shap_values(test_X)
shap.summary_plot(shap_values, test_X)

In [ ]:
xgb.plot_importance(xgb_model) #SHAP value both direction

In [ ]:
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()